In [1]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels, check_pairwise_arrays
from sklearn.metrics.pairwise import rbf_kernel as sklearn_kernel
from scipy.stats import norm, uniform
import casadi as cas
from rkhs_utils.utils import cas_rkhs_norm_squared, cas_rkhs_func_exp, cas_rkhs_func, mmd_eps
np.set_printoptions(suppress=True)

ModuleNotFoundError: No module named 'rkhs_utils'

In [ ]:
# Define constraint function

def f_constraint(x, X):
    """
    Constraint function
    
    x: cp.Variable -- decision variable (dim,)
    X: ndarray -- Samples (n_samples, dim)
    """
#     f = cp.exp(0.1 * (x[0, :] + X[:,0])) + x[1, :] + X[:, 1] -10
    
    f = cas.mtimes(X, x).T - 50

    return f

In [ ]:
##### Sample/set problem constants
# Define constants and sample scenarios
np.random.seed(1)
dim = 1
n_samples = 20
c = np.random.randint(1, 50, size=dim)
X = uniform.rvs(loc=1, scale=9, size=(n_samples, dim))
X = norm.rvs(loc=5, scale=3, size=(n_samples, dim))
M = 100000
alpha = 0.05
epsilon = mmd_eps(n_samples, alpha=alpha)
print(epsilon)
sigma = 0.5

In [ ]:
# create model and decision variables
opti = cas.Opti()
opti.solver('ipopt')
x = opti.variable(dim)
w = opti.variable(n_samples)
t = opti.variable()
s = opti.variable()
obj = cas.dot(c, x)
opti.minimize(-cas.dot(c, x))

# Add constraints
opti.subject_to(cas_rkhs_func_exp(w, X, sigma=sigma) + epsilon * s <= t * alpha)
opti.subject_to(cas_rkhs_norm_squared(w, X, sigma=sigma) <= s**2)
opti.subject_to(f_constraint(x, X) + t <= cas_rkhs_func(w, X, X, sigma=sigma))
opti.subject_to(0 <= cas_rkhs_func(w, X, X, sigma=sigma))
opti.subject_to(s >= 0)
sol = opti.solve()

In [ ]:
print("Objective: ", sol.value(obj))
print("x: ", sol.value(x))
print("w: ", sol.value(w))